In [1]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=fdcbdc0c12fa58c81dacbf81fc5fa5ef73aa877fe08df49a4433a237049b1dc5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [4]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

In [3]:
import pandas as pd

In [5]:
def read_file(file_name):
    import pandas as pd
    
    # Give the headers of the file
    CMPLheader = ['CMPLID', 'ODINO', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'CRASH', 'FAILDATE', \
                  'FIRE', 'INJURED', 'DEATHS', 'COMPDESC', 'CITY', 'STATE', 'VIN', 'DATEA', 'LDATE', \
                  'MILES', 'OCCURENCES', 'CDESCR', 'CMPL_TYPE', 'POLICE_RPT_YN', 'PURCH_DT', \
                  'ORIG_OWNER_YN', 'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'NUM_CYLS', 'DRIVE_TRAIN', \
                  'FUEL_SYS', 'FUEL_TYPE', 'TRANS_TYPE', 'VEH_SPEED', 'DOT', 'TIRE_SIZE', 'LOC_OF_TIRE', \
                  'TIRE_FAIL_TYPE', 'ORIG_EQUIP_YN', 'MANUF_DT', 'SEAT_TYPE', 'RESTRAINT_TYPE', \
                  'DEALER_NAME', 'DEALER_TEL', 'DEALER_CITY', 'DEALER_STATE', 'DEALER_ZIP', 'PROD_TYPE', \
                  'REPAIRED_YN', 'MEDICAL_ATTN', 'VEHICLES_TOWED_YN']
    
    # read the data
    df_Full = pd.read_csv(file_name, sep='\t',encoding='ISO-8859-1', low_memory=False, header=None, \
                     names=CMPLheader)
    
    return df_Full

In [6]:
file_name = 'FLAT_CMPL.txt'
full_data_frame = read_file(file_name)

In [7]:
full_data_frame.head()

,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,DEATHS,COMPDESC,CITY,STATE,VIN,DATEA,LDATE,MILES,OCCURENCES,CDESCR,CMPL_TYPE,POLICE_RPT_YN,PURCH_DT,ORIG_OWNER_YN,ANTI_BRAKES_YN,CRUISE_CONT_YN,NUM_CYLS,DRIVE_TRAIN,FUEL_SYS,FUEL_TYPE,TRANS_TYPE,VEH_SPEED,DOT,TIRE_SIZE,LOC_OF_TIRE,TIRE_FAIL_TYPE,ORIG_EQUIP_YN,MANUF_DT,SEAT_TYPE,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN
0,1,958173.0,Ford Motor Company,LINCOLN,TOWN CAR,1994.0,Y,19941222.0,N,0.0,0.0,"SERVICE BRAKES, HYDRAULIC:PEDALS AND LINKAGES",HIGH LAND PA,MI,1LNLM82W8RY,19950103,19950103,NaN,1.0,BRAKE PEDAL PUSH ROD RETAINER WAS NOT PROPERLY...,EVOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,NaN,NaN
1,2,958146.0,General Motors LLC,GMC,SONOMA,1995.0,NaN,19941215.0,N,0.0,0.0,"SERVICE BRAKES, HYDRAULIC:FOUNDATION COMPONENTS",MOBILE,AL,1GTCS19W3S8,19950103,19950103,NaN,NaN,"VEHICLE STALLS AT HIGH SPEED, RESULTING IN LOS...",EVOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,NaN,NaN
2,3,958127.0,Ford Motor Company,FORD,RANGER,1994.0,NaN,NaN,N,0.0,0.0,ENGINE AND ENGINE COOLING:EXHAUST SYSTEM,N. LAUDERDAL,FL,NaN,19950103,19950103,NaN,NaN,EXHAUST SYSTEM FAILS; PLEASE DESCRIBE DETAILS....,EVOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,NaN,NaN
3,4,958170.0,Ford Motor Company,MERCURY,COUGAR,1995.0,NaN,19950101.0,N,0.0,0.0,"SERVICE BRAKES, HYDRAULIC:FOUNDATION COMPONENTS",CORRAL SPRIN,FL,1MELM62W5SH,19950103,19950103,NaN,1.0,BRAKING SYSTEM FAILURE WITHOUT ABS BRAKES. TT,EVOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,NaN,NaN
4,5,958149.0,"Nissan North America, Inc.",NISSAN,MAXIMA,1987.0,NaN,19941223.0,N,0.0,0.0,VISIBILITY:SUN ROOF ASSEMBLY,COLUMBUS,OH,JN1HU11P3HX,19950103,19950103,NaN,1.0,VEHICLES SUN ROOF GLASS FLEW OFF WHILE DRIVING...,EVOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,NaN,NaN


In [8]:
# extract a specific column from the dataframe
def specific_column(dataframe,column_name):
    return dataframe.loc[:,[column_name]]

In [9]:
column_name = "CDESCR"
complaints_description = specific_column(full_data_frame,column_name)
print(complaints_description.head())
print(len(complaints_description))

                                              CDESCR
0  BRAKE PEDAL PUSH ROD RETAINER WAS NOT PROPERLY...
1  VEHICLE STALLS AT HIGH SPEED, RESULTING IN LOS...
2  EXHAUST SYSTEM FAILS; PLEASE DESCRIBE DETAILS....
3     BRAKING SYSTEM FAILURE WITHOUT ABS BRAKES.  TT
4  VEHICLES SUN ROOF GLASS FLEW OFF WHILE DRIVING...
369171


In [10]:
# extract string at specific index (), as at some elements, the instance is not a string hence we return a blank string 
def extract_string(dataframe,column_name,i):
    to_return = dataframe.loc[i,column_name]
    if(isinstance(to_return,str)):
        return to_return
    else:
        return ""

In [11]:
# index 200
print(200)
print(extract_string(complaints_description,column_name,200))

# index 400
print(400)
print(extract_string(complaints_description,column_name,400))

200
404-261-7770,WK NO., CRUISE CONTROL STUCK AND VEHICLE WENT OPEN THROTTLE POSITION LOST CONTROL HIT ENBANKMENT IMPACT 12:00 POSITION SPEED UNKNOWN.  TT
400
REPLACED TIE ROD END/DRAG LINK.   *AK


In [12]:
#finds the next word in the given sentence given the start index (from where to start looking for words)
# returns the word, the end index of the current word and start index of next word
# if a word is not possible, returns -1 as both end and next_start indices

def find_next_word(sentence,index):
    word = ""
    i = index
    strlen = len(sentence)
    
    if(index >= strlen):
        return (word,-1,-1)
    
    while(i<strlen):
        if(sentence[i]==' ' or sentence[i]==',' or sentence[i]=='.' or sentence[i]=='/' or sentence[i]==';' or sentence[i]==':' or sentence[i]=='?' or sentence[i]=='!'):
            next_start = i
            while(next_start<strlen and (sentence[next_start]==' ' or sentence[next_start]==',' or sentence[next_start]=='.' or sentence[next_start]=='/' or sentence[next_start]==';' or sentence[next_start]==':'  or sentence[next_start]=='?' or sentence[next_start]=='!')):
                next_start = next_start+1
            #print(word,next_start)
            return word,i,next_start
        
        word = word + sentence[i]
        i = i+1
    
    # the string has ended
    return word,i,i

# translate function in python 

In [13]:
# this returns the last word in the sentence 

def last_word(sentence):
    if(sentence==""):
        return ""
    start = 0
    word,end,start = find_next_word(sentence,start)
    prev_word = word
    while(True):
        word,end,start = find_next_word(sentence,start)
        if(start==-1):
            return prev_word
        prev_word = word
    return word

In [14]:
import re

# from the sentence
# removes the leading and trailing whitespaces
# removes the stopwords
# removes double spaces from the sentence

def clean_string(sentence,
                stopwords = set(["*TR","*AK","000","*JB","*NM","TT","*JS","*SLC","NLM","*YH","*TT","*LA","*KB","*CN","YH","ON",
                            "*SD","MJS","*ML","TS","*MJS","*SC","*BF","*SKD","*JG","ME","*YC","DT","18*JB","*TC","*YD","*DSH",
                            "2017*CN","2017*JS","*SCC","*CB","*MR","*DH","*PH","*LN","*AW","*DT*JB","*SMD","*ET","SCC","10*JB",
                            "PH","*LJ","15*JB","2015*JS","15*CN","*TS","*TW","IN","*DT","16*LJ","*AS","*MW","14*CN","*TGW","*CJ","15*LJ",
                            "¿*JB","ET","14*LJ","13*LJ","*TT*JB","*BF*JB","12*LJ","##","*CJS","2017*CT","*PM","11*JB","*MJ","8JB","CJ","*SB",
                            "17*LJ","*DL","*DSY","*LN*JB","(QCAW)","DJR","07*TR","*TR*JB","11*LJ","JB","*CW","QCAW","13*CN","*AKI",
                            "*NJ","KMJ","*CC","09*JB","NM","RK","¿","ME!","ME","*","2013*JS","*TA","¿)*JB","*TP","*SS","","15*JS","CB",
                            "16*JB","JS","T","DO?*AK","\"*AK","*A","TW*","CA","17*CN","**AK","#","2016*JS","(MJS)","*PM*JB","14*JB",
                            "*DT*JB*DT*JB","TX","YD","M","07*JB","*LD","*AKY","NE","JG","10*BF","*K","TR","A","TR","*AL","MGM",
                            "LJ","AZ","AW","PA","FL","GA","2012*JS","***NAR***","-*NM","*NM","08*JB","MO","*ADJ","SC","NJ","JO","11*CW","12*JB",
                            "LI","*AK*JB","OHIO","2014*JS","YRS","NC","19*JB","17*JS","2014*LJ","17*JB","*JB*JB","217*CN","SB","**JB","*AKJLG",
                            "*MJJ","*EH","*B",""])
                ):
    sentence = re.sub(".  ",". ",sentence)
    #sentence = sentence.strip()
    new_sentence = ""
        
    start = 0
    word_start = 0
    word,word_end,start = find_next_word(sentence,start)
    while(start != -1):
        if word not in stopwords:
            if(start != -1):
                new_sentence += sentence[word_start:start]
            else:
                new_sentence += sentence[word_start:len(sentence)]
        word_start = start
        word,word_end,start = find_next_word(sentence,start)
        
    new_sentence = new_sentence.strip()
    return new_sentence

In [15]:
print(extract_string(complaints_description,column_name,200201))
print(clean_string(extract_string(complaints_description,column_name,200201)))

AUTOMATIC SHOULDER RESTRAINT COMES UNDONE ON THE PASSENGER' SIDE FOR NO REASON. PROBLEM WAS REPORTED TO THE DEALERSHIP.  PROBLEM HAS NOT BEEN CORRECTED. *AK
AUTOMATIC SHOULDER RESTRAINT COMES UNDONE THE PASSENGER' SIDE FOR NO REASON. PROBLEM WAS REPORTED TO THE DEALERSHIP. PROBLEM HAS NOT BEEN CORRECTED.


In [16]:
# edits the strings in the dataframe 

def clean_dataframe_strings(dataframe,column_name,start_range,end_range):
    for i in range(start_range,end_range):
        if(i%1000==0):
            print(i)
        sentence = extract_string(dataframe,column_name,i)
        sentence = clean_string(sentence)
        dataframe.at[i,column_name] = sentence

In [18]:
clean_dataframe_strings(complaints_description,"CDESCR",0,100000)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [19]:
nlp = spacy.load("en")

In [25]:
texts = []
for i in range(50000):
  if(i%1000==0):
    print(i)
  
  sentence = extract_string(complaints_description,column_name,i)

  article = []
  doc = nlp(sentence)

  for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        # we add the lematized version of the word
        article.append(w.lemma_.lower())

  texts.append(article)

0
1000
2000
3000
4000
5000
6000
7000
8000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [27]:
texts[24241]

['driving', 'stop', 'truck', 'accelerate']

In [28]:
# converting to bag of words

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [29]:
corpus

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(2, 1),
  (11, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(4, 1), (21, 1), (22, 1), (23, 1), (24, 1)],
 [(2, 1), (24, 1), (25, 1), (26, 1), (27, 1)],
 [(28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)],
 [(2, 1),
  (11, 1),
  (17, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1)],
 [(2, 1),
  (11, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(2, 1),
  (11, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(20, 1),
  (26, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1)],
 [(24, 1), (25, 1), (27, 1), (55, 1)],
 [(4, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59

In [30]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [31]:
ldamodel.show_topics()

[(0,
  '0.043*"fire" + 0.038*"ignition" + 0.038*"engine" + 0.035*"switch" + 0.032*"fuel" + 0.031*"electrical" + 0.030*"vehicle" + 0.027*"short" + 0.026*"replaced" + 0.023*"smoke"'),
 (1,
  '0.114*"seat" + 0.078*"belt" + 0.051*"door" + 0.038*"driver" + 0.031*"passenger" + 0.026*"buckle" + 0.026*"describe" + 0.021*"lock" + 0.020*"child" + 0.019*"latch"'),
 (2,
  '0.097*"brake" + 0.063*"abs" + 0.059*"system" + 0.045*"failure" + 0.030*"pedal" + 0.028*"light" + 0.026*"stop" + 0.024*"lock" + 0.021*"floor" + 0.019*"braking"'),
 (3,
  '0.057*"control" + 0.045*"fail" + 0.043*"vehicle" + 0.036*"accident" + 0.035*"causing" + 0.029*"recall" + 0.022*"loss" + 0.019*"work" + 0.016*"cause" + 0.016*"prior"'),
 (4,
  '0.069*"describe" + 0.063*"vehicle" + 0.053*"fire" + 0.045*"steering" + 0.043*"consumer" + 0.036*"car" + 0.029*"caught" + 0.024*"parked" + 0.023*"column" + 0.023*"drive"'),
 (5,
  '0.143*"brake" + 0.037*"rear" + 0.026*"replaced" + 0.024*"vehicle" + 0.023*"describe" + 0.022*"fail" + 0.021*"r

In [32]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.127913  0.210709       1        1  12.753355
0     -0.139502 -0.157721       2        1  12.216068
2     -0.188108  0.239272       3        1  11.703800
1      0.230341  0.074495       4        1  11.290993
4     -0.075981 -0.229180       5        1  10.425027
6     -0.016558 -0.098384       6        1   8.815831
9      0.059293 -0.138111       7        1   8.784821
7      0.317103  0.073965       8        1   8.603921
3     -0.071697  0.021637       9        1   8.095331
8      0.013022  0.003319      10        1   7.310852, topic_info=              Term          Freq         Total Category  logprob  loglift
2            brake  12701.000000  12701.000000  Default  30.0000  30.0000
93            seat   6166.000000   6166.000000  Default  29.0000  29.0000
56             air   3174.000000   3174.000000  Default  28.0000  28.0000
34            belt   4356.000000   4356.000000  Default  27.0000  27.0000
102   transmission   2731.000000   2731.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
173           fall    223.711183    983.190820  Topic10  -4.9296   1.1354
1153         parts    208.712322    499.022836  Topic10  -4.9990   1.7441
217            car    237.226381   2106.504472  Topic10  -4.8710   0.4320
326       consumer    221.291740   2230.322694  Topic10  -4.9405   0.3054
212         failed    216.345308   1739.713949  Topic10  -4.9631   0.5312

[633 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1385      6  0.997835       
7030      4  0.014689       
7030     10  0.976828       
395       1  0.022237  12:00
395       7  0.187780  12:00
...     ...       ...    ...
148       9  0.819702   work
148      10  0.149267   work
1123      1  0.997249   worn
1168      5  0.958198  wrong
1168      6  0.030259  wrong

[1446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 2, 5, 7, 10, 8, 4, 9])

In [33]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.127913  0.210709       1        1  12.752815
0     -0.139502 -0.157721       2        1  12.212400
2     -0.188108  0.239272       3        1  11.703093
1      0.230341  0.074495       4        1  11.293667
4     -0.075981 -0.229180       5        1  10.421185
6     -0.016558 -0.098384       6        1   8.812468
9      0.059293 -0.138111       7        1   8.789508
7      0.317103  0.073965       8        1   8.608501
3     -0.071697  0.021637       9        1   8.092242
8      0.013022  0.003319      10        1   7.314121, topic_info=              Term          Freq         Total Category  logprob  loglift
2            brake  12701.000000  12701.000000  Default  30.0000  30.0000
93            seat   6168.000000   6168.000000  Default  29.0000  29.0000
56             air   3176.000000   3176.000000  Default  28.0000  28.0000
34            belt   4357.000000   4357.000000  Default  27.0000  27.0000
102   transmission   2732.000000   2732.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
173           fall    223.811204    983.148693  Topic10  -4.9296   1.1354
1153         parts    208.805637    499.183420  Topic10  -4.9990   1.7438
217            car    237.332445   2106.163803  Topic10  -4.8710   0.4322
326       consumer    221.390679   2229.752320  Topic10  -4.9405   0.3056
212         failed    216.442036   1739.614360  Topic10  -4.9631   0.5313

[633 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1385      6  0.998214       
7030      4  0.014683       
7030     10  0.976397       
395       1  0.022226  12:00
395       7  0.187683  12:00
...     ...       ...    ...
148       9  0.819905   work
148      10  0.149304   work
1123      1  0.997291   worn
1168      5  0.958545  wrong
1168      6  0.030270  wrong

[1446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 2, 5, 7, 10, 8, 4, 9])

In [34]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.127913  0.210709       1        1  12.757818
0     -0.139502 -0.157721       2        1  12.212508
2     -0.188108  0.239272       3        1  11.696755
1      0.230341  0.074495       4        1  11.294642
4     -0.075981 -0.229180       5        1  10.419793
6     -0.016558 -0.098384       6        1   8.817334
9      0.059293 -0.138111       7        1   8.789124
7      0.317103  0.073965       8        1   8.606004
3     -0.071697  0.021637       9        1   8.092846
8      0.013022  0.003319      10        1   7.313177, topic_info=              Term          Freq         Total Category  logprob  loglift
2            brake  12701.000000  12701.000000  Default  30.0000  30.0000
93            seat   6168.000000   6168.000000  Default  29.0000  29.0000
56             air   3175.000000   3175.000000  Default  28.0000  28.0000
34            belt   4357.000000   4357.000000  Default  27.0000  27.0000
102   transmission   2731.000000   2731.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
173           fall    223.782310    983.354009  Topic10  -4.9296   1.1352
1153         parts    208.778680    499.182486  Topic10  -4.9990   1.7438
217            car    237.301805   2105.905402  Topic10  -4.8710   0.4323
326       consumer    221.362097   2229.474726  Topic10  -4.9405   0.3058
212         failed    216.414093   1739.732048  Topic10  -4.9631   0.5312

[633 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1385      6  0.997665       
7030      4  0.014685       
7030     10  0.976519       
395       1  0.022231  12:00
395       7  0.187726  12:00
...     ...       ...    ...
148       9  0.819862   work
148      10  0.149296   work
1123      1  0.996902   worn
1168      5  0.958655  wrong
1168      6  0.030273  wrong

[1446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 2, 5, 7, 10, 8, 4, 9])

In [35]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.127913  0.210709       1        1  12.756722
0     -0.139502 -0.157721       2        1  12.214038
2     -0.188108  0.239272       3        1  11.697083
1      0.230341  0.074495       4        1  11.292409
4     -0.075981 -0.229180       5        1  10.425875
6     -0.016558 -0.098384       6        1   8.814931
9      0.059293 -0.138111       7        1   8.789157
7      0.317103  0.073965       8        1   8.607726
3     -0.071697  0.021637       9        1   8.094913
8      0.013022  0.003319      10        1   7.307146, topic_info=              Term          Freq         Total Category  logprob  loglift
2            brake  12701.000000  12701.000000  Default  30.0000  30.0000
93            seat   6167.000000   6167.000000  Default  29.0000  29.0000
56             air   3176.000000   3176.000000  Default  28.0000  28.0000
34            belt   4356.000000   4356.000000  Default  27.0000  27.0000
102   transmission   2729.000000   2729.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
173           fall    223.597756    983.083815  Topic10  -4.9296   1.1355
1153         parts    208.606500    498.953089  Topic10  -4.9990   1.7443
217            car    237.106101   2106.632068  Topic10  -4.8710   0.4320
326       consumer    221.179539   2230.388584  Topic10  -4.9405   0.3054
212         failed    216.235615   1739.574546  Topic10  -4.9631   0.5313

[633 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1385      6  0.997936       
7030      4  0.014696       
7030     10  0.977311       
395       1  0.022227  12:00
395       7  0.187696  12:00
...     ...       ...    ...
148       9  0.819793   work
148      10  0.149283   work
1123      1  0.996988   worn
1168      5  0.958124  wrong
1168      6  0.030257  wrong

[1446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 2, 5, 7, 10, 8, 4, 9])

In [36]:
ldamodel_20 = LdaModel(corpus=corpus, num_topics=20, id2word=dictionary)

In [37]:
pyLDAvis.gensim.prepare(ldamodel_20, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.160933  0.153446       1        1  12.329957
9      0.070981 -0.150117       2        1   7.522119
3      0.190461  0.087683       3        1   7.095995
8      0.173939  0.160393       4        1   6.641218
14     0.115766  0.051584       5        1   5.735091
17    -0.109461  0.148486       6        1   5.157246
4     -0.142219  0.165103       7        1   5.071915
13     0.038373 -0.008595       8        1   4.674867
15     0.088785 -0.182573       9        1   4.500261
12     0.105183 -0.017808      10        1   4.257653
18     0.020911 -0.148686      11        1   4.071373
11    -0.140003 -0.080943      12        1   4.063852
7     -0.085670  0.010467      13        1   3.859730
10    -0.148555 -0.057160      14        1   3.835708
2      0.129051 -0.175342      15        1   3.827279
6     -0.159661 -0.013237      16        1   3.811247
1     -0.072146  0.032002      17        1   3.768863
16     0.139944  0.169513      18        1   3.561778
5      0.113135  0.007111      19        1   3.337805
19    -0.167882 -0.151325      20        1   2.876043, topic_info=          Term          Freq         Total Category  logprob  loglift
2        brake  13062.000000  13062.000000  Default  30.0000  30.0000
93        seat   6088.000000   6088.000000  Default  29.0000  29.0000
45        rear   5261.000000   5261.000000  Default  28.0000  28.0000
34        belt   4364.000000   4364.000000  Default  27.0000  27.0000
354       fire   4048.000000   4048.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
4         fail    855.616978   6921.215147  Topic20  -2.6552   1.4582
212     failed    301.675163   1826.491330  Topic20  -3.6977   1.7480
100   replaced    379.568098   4721.025814  Topic20  -3.4680   1.0280
1379      neck    103.022498    163.847965  Topic20  -4.7721   3.0848
240       idle     89.881530    237.908175  Topic20  -4.9086   2.5754

[1067 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1385     14  0.995172       
7030      2  0.025058       
7030      9  0.125292       
7030     19  0.843636       
182      11  0.963385  10mph
...     ...       ...    ...
148      10  0.288469   work
148      11  0.116761   work
148      19  0.020605   work
1168      7  0.060164  wrong
1168     15  0.932544  wrong

[2233 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 10, 4, 9, 15, 18, 5, 14, 16, 13, 19, 12, 8, 11, 3, 7, 2, 17, 6, 20])

In [38]:
ldamodel_7 = LdaModel(corpus=corpus, num_topics=7, id2word=dictionary)

In [39]:
pyLDAvis.gensim.prepare(ldamodel_7, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.242419  0.001313       1        1  18.874577
6      0.037356 -0.225665       2        1  18.502975
5     -0.221276  0.057589       3        1  15.171631
4      0.123174  0.154475       4        1  13.539132
0      0.158603  0.195216       5        1  11.967308
1      0.007661  0.026728       6        1  11.192916
3      0.136902 -0.209657       7        1  10.751461, topic_info=         Term          Freq         Total Category  logprob  loglift
2       brake  11833.000000  11833.000000  Default  30.0000  30.0000
93       seat   5821.000000   5821.000000  Default  29.0000  29.0000
19   steering   3476.000000   3476.000000  Default  28.0000  28.0000
354      fire   5105.000000   5105.000000  Default  27.0000  27.0000
34       belt   4135.000000   4135.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
250     loose    248.571874    710.597054   Topic7  -5.2100   1.1798
142    caused    253.345698   1262.562537   Topic7  -5.1909   0.6240
350    caught    249.570595   1399.229615   Topic7  -5.2059   0.5062
73     engine    259.986124   3353.683881   Topic7  -5.1651  -0.3271
171     cause    255.965681   3078.606704   Topic7  -5.1806  -0.2571

[487 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1385      3  0.995492         
395       2  0.996048    12:00
1700      3  0.992881    30mph
825       2  0.994672    35mph
872       7  0.987131    55mph
...     ...       ...      ...
668       4  0.001220   wiring
668       6  0.008539   wiring
672       4  0.997316    won't
1124      4  0.994857  working
1123      1  0.996186     worn

[1054 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 6, 5, 1, 2, 4])

In [40]:
pyLDAvis.gensim.prepare(ldamodel_7, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.242419  0.001313       1        1  18.882655
6      0.037356 -0.225665       2        1  18.506003
5     -0.221276  0.057589       3        1  15.159196
4      0.123174  0.154475       4        1  13.541465
0      0.158603  0.195216       5        1  11.971694
1      0.007661  0.026728       6        1  11.188711
3      0.136902 -0.209657       7        1  10.750276, topic_info=         Term          Freq         Total Category  logprob  loglift
2       brake  11832.000000  11832.000000  Default  30.0000  30.0000
93       seat   5822.000000   5822.000000  Default  29.0000  29.0000
19   steering   3476.000000   3476.000000  Default  28.0000  28.0000
354      fire   5106.000000   5106.000000  Default  27.0000  27.0000
34       belt   4135.000000   4135.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
250     loose    248.544473    710.703655   Topic7  -5.2100   1.1796
142    caused    253.317771   1262.332762   Topic7  -5.1909   0.6242
350    caught    249.543085   1399.390587   Topic7  -5.2059   0.5061
73     engine    259.957465   3353.885021   Topic7  -5.1651  -0.3271
171     cause    255.937465   3078.056270   Topic7  -5.1806  -0.2569

[487 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1385      3  0.996305         
395       2  0.995886    12:00
1700      3  0.993688    30mph
825       2  0.994511    35mph
872       7  0.987239    55mph
...     ...       ...      ...
668       4  0.001221   wiring
668       6  0.008546   wiring
672       4  0.997145    won't
1124      4  0.994687  working
1123      1  0.995763     worn

[1054 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 6, 5, 1, 2, 4])